In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load your transactions data
articles_df = pd.read_csv('./dataset/articles.csv')
customers_df = pd.read_csv('./dataset/customers.csv')
training_transactions = pd.read_csv('./dataset/split/fold_0/train.csv')
validation_transactions = pd.read_csv('./dataset/split/fold_0/test.csv')
# Load your transactions data

print("articles_df: ", articles_df.columns)
print("customers_df: ", customers_df.columns)
print("transactions_df: ", training_transactions.columns)

articles_df:  Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')
customers_df:  Index(['customer_id', 'FN', 'Active', 'club_member_status',
       'fashion_news_frequency', 'age', 'postal_code'],
      dtype='object')
transactions_df:  Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'], dtype='object')


In [3]:
# # Step 2: Positive Samples
# # Group by customer and list all articles they have purchased
# positive_samples = training_transactions.groupby('customer_id')['article_id'].agg(list).reset_index()

# # All unique articles
# all_articles = set(articles_df['article_id'].astype(str))

# # Step 4: Compile Training Data
# # Format data as sentences: each user has two lists, one for positives, one for negatives
# training_data = []
# for _, row in positive_samples.iterrows():
#     training_data.append(row['article_id'])  # Positive sentence
#     # training_data.append(row['negative_samples'])  # Negative sentence

# # Now training_data can be used for embedding training


KeyboardInterrupt: 

In [ ]:
# import warnings
# warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
# warnings.filterwarnings("ignore", message="numpy.dtype size changed")
# warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
# import gensim
# assert gensim.models.word2vec.FAST_VERSION > -1
# from collections import Counter
# import random

# for purchase in training_data:
#     random.shuffle(purchase)
    
# print(len(training_data))

# # Calculate the length of each sublist
# lengths = [len(sublist) for sublist in training_data]

# # Count the frequency of each length
# length_distribution = Counter(lengths)


In [ ]:
# import os
# import time
# from gensim.models import Word2Vec

# def train_and_save_models(sentences, vector_sizes, epochs=10, min_count=10, workers=8, sg=1, hs=0, negative=5, window=9999, save_path='./model'):
#     results = []
    
#     # Ensure the save directory exists
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)
#         print(f"Directory {save_path} created.")
    
#     for vector_size in vector_sizes:
#         start = time.time()
        
#         # Train the model
#         model = Word2Vec(
#             sentences=sentences,
#             epochs=epochs,
#             min_count=min_count,
#             vector_size=vector_size,
#             workers=workers,
#             sg=sg,
#             hs=hs,
#             negative=negative,
#             window=window
#         )
        
#         # Calculate training duration
#         duration = time.time() - start
        
#         # Save the model
#         model_save_path = f"{save_path}/item2vec_{vector_size}.model"
#         model.save(model_save_path)
        
#         # Record results
#         training_loss = model.get_latest_training_loss()
#         results.append((vector_size, duration, training_loss, model_save_path))
        
#         print(f"Model with vector_size {vector_size} saved to {model_save_path}. Time passed: {duration:.2f} seconds.")

#     return results

# # Example usage:
# vector_sizes = range(96, 257, 32)  # From 96 to 256 with a step of 32
# train_and_save_models(training_data, vector_sizes)


In [10]:
import numpy as np
import torch
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec
assert gensim.models.word2vec.FAST_VERSION > -1
from torch import nn
from sklearn.preprocessing import normalize
import faiss
from datetime import timedelta

# Ensure 't_dat' is a datetime object
training_transactions['t_dat'] = pd.to_datetime(training_transactions['t_dat'])

# Compute the current date and last week's start date
current_date = training_transactions['t_dat'].max()
last_week_start = current_date - timedelta(days=14)
recent_transactions = training_transactions[(training_transactions['t_dat'] > last_week_start)]

# Define function to calculate user profiles
def calculate_user_profile(item_ids, model):
    vectors = []
    for item_id in item_ids:
        if item_id in model.wv.key_to_index:  # Check if the item is in the model's vocabulary
            vectors.append(model.wv[item_id])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Compute MAP@K for recommendation
def compute_map_k(actual, predicted, k=12):
    actual_set = set(actual)
    predicted = predicted[:k]
    hits = [1 if item in actual_set else 0 for item in predicted]
    cumulative_hits = np.cumsum(hits)
    precision_at_k = [hits[i] * cumulative_hits[i] / (i + 1) for i in range(len(hits))]
    return sum(precision_at_k) / min(len(actual_set), k) if actual_set else 0

# Function to create and query a FAISS index for popularity recommendations
def create_faiss_index(user_profiles):
    user_profiles_array = np.stack(user_profiles.values).astype('float32')
    user_profiles_array = normalize(user_profiles_array)
    index = faiss.IndexFlatL2(user_profiles_array.shape[1])
    gpu_resources = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(gpu_resources, 0, index)
    gpu_index.add(user_profiles_array)
    k = 200  # Number of nearest neighbors
    distances, indices = gpu_index.search(user_profiles_array, k)
    return distances, indices

# Function to process batches and get similarity recommendations
def process_batch(batch_user_vectors, item_vectors_norm, item_ids, top_k=100):
    batch_user_vectors_norm = batch_user_vectors / batch_user_vectors.norm(dim=1, keepdim=True)
    similarities = torch.mm(batch_user_vectors_norm, item_vectors_norm.t())
    top_indices = torch.topk(similarities, top_k, dim=1).indices
    return top_indices

# Function to aggregate recommendations based on nearest neighbors using pre-filtered transactions
def aggregate_recommendations(user_profiles, indices, recent_transactions, k=12):
    # Group transactions by customer_id and aggregate article counts
    grouped_transactions = recent_transactions.groupby('customer_id')['article_id'].agg(lambda x: x.value_counts().to_dict()).to_dict()
    
    user_id_list = user_profiles.index.tolist()
    recommendations = {}
    
    # Use tqdm to show the progress bar
    for i in tqdm(range(len(user_id_list)), desc="Aggregating Recommendations"):
        user_id = user_id_list[i]
        article_counts = {}
        for idx in indices[i]:
            neighbor_id = user_id_list[idx]
            if neighbor_id in grouped_transactions:
                for article_id, count in grouped_transactions[neighbor_id].items():
                    article_counts[article_id] = article_counts.get(article_id, 0) + count
        
        # Sort articles by their aggregated counts and select the top_k articles
        sorted_items = sorted(article_counts.items(), key=lambda item: item[1], reverse=True)
        recommendations[user_id] = [item[0] for item in sorted_items[:k]]

    return recommendations

# Updated main evaluation function integrating both methods
def evaluate_model(model, transactions_df, validation_transactions, device='cpu', batch_size=4096):
    user_items = transactions_df.groupby('customer_id')['article_id'].apply(list)
    user_profiles = user_items.apply(lambda x: calculate_user_profile(x, model))
    
    # Prepare tensors for similarity-based recommendations
    item_ids = list(model.wv.index_to_key)
    item_vectors = torch.tensor([model.wv[item] for item in item_ids], dtype=torch.float).to(device)
    item_vectors_norm = item_vectors / item_vectors.norm(dim=1, keepdim=True)
    user_ids = list(user_profiles.index)
    user_vectors = torch.tensor(list(user_profiles.values), dtype=torch.float).to(device)
    
    # Compute similarity recommendations
    similarity_recommendations = {}
    for start_idx in tqdm(range(0, len(user_vectors), batch_size)):
        end_idx = min(start_idx + batch_size, len(user_vectors))
        batch_user_vectors = user_vectors[start_idx:end_idx]
        top_indices = process_batch(batch_user_vectors, item_vectors_norm, item_ids)
        for row_index, i in enumerate(range(start_idx, end_idx)):
            user_id = user_ids[i]
            recommendations = [item_ids[idx] for idx in top_indices[row_index].cpu().tolist()]
            similarity_recommendations[user_id] = recommendations
    
    # Compute popularity recommendations using FAISS
    distances, indices = create_faiss_index(user_profiles)
    popularity_recommendations = aggregate_recommendations(user_profiles, indices, recent_transactions)

    # Prepare actual purchases data for MAP calculation
    actual_purchases = validation_transactions.groupby('customer_id')['article_id'].agg(list)
    actual_lists = [actual_purchases.loc[user_id] for user_id in actual_purchases.index if user_id in popularity_recommendations]

    # Calculate MAP@12 for both recommendation methods
    predicted_similarity_lists = [similarity_recommendations[user_id] for user_id in actual_purchases.index if user_id in similarity_recommendations]
    predicted_popularity_lists = [popularity_recommendations[user_id] for user_id in actual_purchases.index if user_id in popularity_recommendations]

    similarity_map_scores = [compute_map_k(actual, predicted, 12) for actual, predicted in zip(actual_lists, predicted_similarity_lists)]
    popularity_map_scores = [compute_map_k(actual, predicted, 12) for actual, predicted in zip(actual_lists, predicted_popularity_lists)]

    mean_similarity_map = np.mean(similarity_map_scores)
    mean_popularity_map = np.mean(popularity_map_scores)
    
    return mean_similarity_map, mean_popularity_map


# Usage example
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Word2Vec.load('model\item2vec_96.model')  # Load the model
mean_similarity_map, mean_popularity_map = evaluate_model(model, training_transactions, validation_transactions, device)
print(f"MAP@12 - Similarity: {mean_similarity_map}, Popularity: {mean_popularity_map}")

100%|██████████| 121/121 [00:22<00:00,  5.31it/s]


MAP@12 - Similarity: 0.01960577880033997, Popularity: 0.021528205169237827
